# Lab 7: Missing Values and Cleaning Messy Data

In [199]:
library(tidyverse)

In [200]:
(stocks <- tibble(
  Year   = c(2015, 2015, 2015, 2015, 2016, 2016, 2016),
  Qtr    = c(   1,    2,    3,    4,    2,    3,    4),
  Return = c(1.88, 0.59, 0.35,   NA, 0.92, 0.17, 2.66)
))

Year,Qtr,Return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,NA
2016,2,0.92
2016,3,0.17
2016,4,2.66


## Missing Values
Missing values can be:
1. *Explicit* (marked as NA in our data)
1. *Implicit* (not present in the data)

In this example we have one explicitly missing value for the 4th quarter of 2015. Are there any other missing values? Yes, because we do not have an observation for the first quarter of 2016.

`complete`: Turns implicit missing values into explicit missing values.

In [201]:
?complete

In [202]:
stocks %>% complete(Year, Qtr)

Year,Qtr,Return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,NA
2016,1,NA
2016,2,0.92
2016,3,0.17
2016,4,2.66


`fill`: Fills missing values in selected columns using the previous entry.

In [203]:
stocks %>% complete(Year, Qtr) %>% fill(Return)
# yoou can look up imputation methods

Year,Qtr,Return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,0.35
2016,1,0.35
2016,2,0.92
2016,3,0.17
2016,4,2.66


The missing values also become `explicit` if we widen the tibble.

In [204]:
stocks_wide = stocks %>% pivot_wider(names_from = Year, values_from = Return)

stocks_wide

Qtr,2015,2016
<dbl>,<dbl>,<dbl>
1,1.88,NA
2,0.59,0.92
3,0.35,0.17
4,NA,2.66


`pivot_longer` will keep all these explicitly missing values by default.

In [205]:
stocks_wide %>% pivot_longer(cols = `2015`:`2016`, names_to = 'Year') %>%
arrange(Year)

Qtr,Year,value
<dbl>,<chr>,<dbl>
1,2015,1.88
2,2015,0.59
3,2015,0.35
4,2015,NA
1,2016,NA
2,2016,0.92
3,2016,0.17
4,2016,2.66


## Cleaning the messy data

In [206]:
datacamp_url = "https://assets.datacamp.com/production/repositories/34/datasets/b3c1036d9a60a9dfe0f99051d2474a54f76055ea/weather.rds"
weather = readRDS(url(datacamp_url))

In [207]:
weather %>% head

,X,year,month,measure,X1,X2,X3,X4,X5,X6,⋯,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31
,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,2014,12,Max.TemperatureF,64,42,51,43,42,45,⋯,44,47,46,59,50,52,52,41,30,30
2,2,2014,12,Mean.TemperatureF,52,38,44,37,34,42,⋯,39,45,44,52,44,45,46,36,26,25
3,3,2014,12,Min.TemperatureF,39,33,37,30,26,38,⋯,33,42,41,44,37,38,40,30,22,20
4,4,2014,12,Max.Dew.PointF,46,40,49,24,37,45,⋯,39,45,46,58,31,34,42,26,10,8
5,5,2014,12,MeanDew.PointF,40,27,42,21,25,40,⋯,34,42,44,43,29,31,35,20,4,5
6,6,2014,12,Min.DewpointF,26,17,24,13,12,36,⋯,25,37,41,29,28,29,27,10,-6,1


The first row is for row number, let's ignore that

In [208]:
weather = weather %>% select(-X)

It looks like the values for the weather measurements (column 3) for each day of the month are stored in the columns `X1` to `X31`. From `tidy data` perspective, the data set is messy because:
* column names are values (`X1` to `X31`)
* variable names are represented as values (column 3 - `measure`)

We can correct it by using `pivot_longer`

In [209]:
tidy_weather = weather %>% 
pivot_longer(cols = `X1`:`X31`, names_to = 'day', values_to = "value") %>%
select(year, month, day, everything())

head(tidy_weather)

year,month,day,measure,value
<int>,<int>,<chr>,<chr>,<chr>
2014,12,X1,Max.TemperatureF,64
2014,12,X2,Max.TemperatureF,42
2014,12,X3,Max.TemperatureF,51
2014,12,X4,Max.TemperatureF,43
2014,12,X5,Max.TemperatureF,42
2014,12,X6,Max.TemperatureF,45


In [210]:
tidy_weather %>% select(measure)%>% unique

measure
<chr>
Max.TemperatureF
Mean.TemperatureF
Min.TemperatureF
Max.Dew.PointF
MeanDew.PointF
Min.DewpointF
Max.Humidity
Mean.Humidity
Min.Humidity


The values in the column `measure` of the weather dataset should be variables.

In [211]:
tidy_weather = tidy_weather %>% pivot_wider(names_from = measure, values_from = value)

head(tidy_weather)

year,month,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,⋯,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2014,12,X1,64,52,39,46,40,26,74,⋯,10,10,10,22,13,29,0.01,6,Rain,268
2014,12,X2,42,38,33,40,27,17,92,⋯,10,8,2,24,15,29,0.10,7,Rain-Snow,62
2014,12,X3,51,44,37,49,42,24,100,⋯,10,5,1,29,12,38,0.44,8,Rain,254
2014,12,X4,43,37,30,24,21,13,69,⋯,10,10,10,25,12,33,0.00,3,,292
2014,12,X5,42,34,26,37,25,12,85,⋯,10,10,5,22,10,26,0.11,5,Rain,61
2014,12,X6,45,42,38,45,40,36,100,⋯,10,4,0,22,8,25,1.09,8,Rain,313


In [212]:
tidy_weather %>% head

year,month,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,⋯,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2014,12,X1,64,52,39,46,40,26,74,⋯,10,10,10,22,13,29,0.01,6,Rain,268
2014,12,X2,42,38,33,40,27,17,92,⋯,10,8,2,24,15,29,0.10,7,Rain-Snow,62
2014,12,X3,51,44,37,49,42,24,100,⋯,10,5,1,29,12,38,0.44,8,Rain,254
2014,12,X4,43,37,30,24,21,13,69,⋯,10,10,10,25,12,33,0.00,3,,292
2014,12,X5,42,34,26,37,25,12,85,⋯,10,10,5,22,10,26,0.11,5,Rain,61
2014,12,X6,45,42,38,45,40,36,100,⋯,10,4,0,22,8,25,1.09,8,Rain,313


There are couple of things still odd in this dataset. One of them is name of the days. They start with an `X`. We can fix it by `str_replace` function. In following weeks, we are going to cover it in detail when we cover the strings. Let me quickly remind you the usage:

In [213]:
# Replace $ with nothing, so it removes the dollar sign
cost = c("$8", "12.5$", "$45")
cost = str_replace_all(cost, "\\$", "")
print(cost) 

#change its type to numeric
cost = as.numeric(cost)
print(cost)

[1] "8"    "12.5" "45"  
[1]  8.0 12.5 45.0


#### Exercise 1: Remove `X` from the `day` entries and change its type to `numeric`

In [214]:
tidy_weather = tidy_weather %>% mutate(day = as.integer(str_replace(day, "X", "")))
tidy_weather %>% head

year,month,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,⋯,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2014,12,1,64,52,39,46,40,26,74,⋯,10,10,10,22,13,29,0.01,6,Rain,268
2014,12,2,42,38,33,40,27,17,92,⋯,10,8,2,24,15,29,0.10,7,Rain-Snow,62
2014,12,3,51,44,37,49,42,24,100,⋯,10,5,1,29,12,38,0.44,8,Rain,254
2014,12,4,43,37,30,24,21,13,69,⋯,10,10,10,25,12,33,0.00,3,,292
2014,12,5,42,34,26,37,25,12,85,⋯,10,10,5,22,10,26,0.11,5,Rain,61
2014,12,6,45,42,38,45,40,36,100,⋯,10,4,0,22,8,25,1.09,8,Rain,313


#### Exercise 2: Combine the year, month, and day columns into a new column called date. Hint: Use `unite` function

In [215]:
tidy_weather = tidy_weather %>%
unite(col = date, day, month, year, sep = "-")

tidy_weather %>% head

date,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,Mean.Humidity,Min.Humidity,⋯,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1-12-2014,64,52,39,46,40,26,74,63,52,⋯,10,10,10,22,13,29,0.01,6,Rain,268
2-12-2014,42,38,33,40,27,17,92,72,51,⋯,10,8,2,24,15,29,0.10,7,Rain-Snow,62
3-12-2014,51,44,37,49,42,24,100,79,57,⋯,10,5,1,29,12,38,0.44,8,Rain,254
4-12-2014,43,37,30,24,21,13,69,54,39,⋯,10,10,10,25,12,33,0.00,3,,292
5-12-2014,42,34,26,37,25,12,85,66,47,⋯,10,10,5,22,10,26,0.11,5,Rain,61
6-12-2014,45,42,38,45,40,36,100,93,85,⋯,10,4,0,22,8,25,1.09,8,Rain,313


#### Exercise 3: Move events variable to the second column (Just after the `date`)

In [216]:
tidy_weather = tidy_weather %>% 
select(date, Events, everything())
tidy_weather

date,Events,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,Mean.Humidity,⋯,Min.Sea.Level.PressureIn,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,WindDirDegrees
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1-12-2014,Rain,64,52,39,46,40,26,74,63,⋯,30.01,10,10,10,22,13,29,0.01,6,268
2-12-2014,Rain-Snow,42,38,33,40,27,17,92,72,⋯,30.4,10,8,2,24,15,29,0.10,7,62
3-12-2014,Rain,51,44,37,49,42,24,100,79,⋯,29.87,10,5,1,29,12,38,0.44,8,254
4-12-2014,,43,37,30,24,21,13,69,54,⋯,30.09,10,10,10,25,12,33,0.00,3,292
5-12-2014,Rain,42,34,26,37,25,12,85,66,⋯,30.45,10,10,5,22,10,26,0.11,5,61
6-12-2014,Rain,45,42,38,45,40,36,100,93,⋯,30.16,10,4,0,22,8,25,1.09,8,313
7-12-2014,Rain,38,30,21,36,20,-3,92,61,⋯,30.24,10,10,5,25,15,32,0.13,6,350
8-12-2014,Snow,29,24,18,28,16,3,92,70,⋯,30.51,10,8,2,21,13,28,0.03,8,354
9-12-2014,Rain,49,39,29,49,41,28,100,93,⋯,29.49,10,2,1,38,20,52,2.90,8,38


#### Exercise 4: `PrecipitationIn` has “T”s (Trace). “Traces” in precipitation are defined as less than 0.005 inch. So, in this case, we need to map “T”s to 0.

In [219]:
look = tidy_weather %>% filter(PrecipitationIn == "T")
look %>% head

date,Events,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,Mean.Humidity,⋯,Min.Sea.Level.PressureIn,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,WindDirDegrees
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
12-12-2014,Snow,39,35,31,28,27,25,85,75,⋯,29.81,10,10,7,16,11,21,T,7,286
13-12-2014,,42,37,32,28,26,24,75,65,⋯,29.78,10,10,10,17,12,23,T,5,298
16-12-2014,Rain,44,40,35,42,36,30,96,85,⋯,29.92,10,9,5,8,4,10,T,8,79
20-12-2014,Snow,36,32,27,30,24,20,89,79,⋯,30.17,10,10,7,21,10,26,T,6,350
21-12-2014,Snow,36,33,30,30,27,25,85,77,⋯,30.28,10,9,6,16,9,20,T,8,2
7-1-2015,Snow,26,15,4,12,3,-13,88,58,⋯,29.76,10,9,2,31,16,40,T,3,272


In [220]:
tidy_weather =  tidy_weather %>% 
mutate(PrecipitationIn = ifelse(PrecipitationIn == "T", "0", PrecipitationIn))

In [222]:
test = tidy_weather %>% filter(PrecipitationIn == "T")
test %>% head

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


date,Events,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,Mean.Humidity,⋯,Min.Sea.Level.PressureIn,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,WindDirDegrees
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


#### Exercise 5: Other then events, all measures are numeric but coded as chracter. Convert them to numeric

In [223]:
tidy_weather = tidy_weather %>% mutate_each(funs = list(as.numeric), Max.TemperatureF:WindDirDegrees)
tidy_weather %>% head

date,Events,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,Mean.Humidity,⋯,Min.Sea.Level.PressureIn,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,WindDirDegrees
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1-12-2014,Rain,64,52,39,46,40,26,74,63,⋯,30.01,10,10,10,22,13,29,0.01,6,268
2-12-2014,Rain-Snow,42,38,33,40,27,17,92,72,⋯,30.40,10,8,2,24,15,29,0.10,7,62
3-12-2014,Rain,51,44,37,49,42,24,100,79,⋯,29.87,10,5,1,29,12,38,0.44,8,254
4-12-2014,,43,37,30,24,21,13,69,54,⋯,30.09,10,10,10,25,12,33,0.00,3,292
5-12-2014,Rain,42,34,26,37,25,12,85,66,⋯,30.45,10,10,5,22,10,26,0.11,5,61
6-12-2014,Rain,45,42,38,45,40,36,100,93,⋯,30.16,10,4,0,22,8,25,1.09,8,313


#### Exercise 6: What are the unique events in the dataset?

In [224]:
tidy_weather %>% select(Events) %>% unique %>% deframe

[1] "Rain"                       "Rain-Snow"                 
 [3] ""                           "Snow"                      
 [5] "Fog-Rain"                   "Fog-Snow"                  
 [7] "Fog-Rain-Snow"              NA                          
 [9] "Fog-Rain-Thunderstorm"      "Fog"                       
[11] "Rain-Thunderstorm"          "Thunderstorm"              
[13] "Fog-Rain-Hail-Thunderstorm"

#### Exercise 7: Empty means, there is no event. Change it to `Clear`

In [225]:
tidy_weather = tidy_weather %>% 
mutate(Events = ifelse(Events == "", "Clear", Events))

tidy_weather %>% select(Events) %>% unique %>% deframe

[1] "Rain"                       "Rain-Snow"                 
 [3] "Clear"                      "Snow"                      
 [5] "Fog-Rain"                   "Fog-Snow"                  
 [7] "Fog-Rain-Snow"              NA                          
 [9] "Fog-Rain-Thunderstorm"      "Fog"                       
[11] "Rain-Thunderstorm"          "Thunderstorm"              
[13] "Fog-Rain-Hail-Thunderstorm"

Reference: [Cleaning Messy Weather Dataset with tidyverse](https://www.rpubs.com/justinhtet/cleaning-messy-weather-dataset-with-tidyverse)